In [68]:
import random
from collections import deque
import tkinter as tk
from tkinter import messagebox


class CodebreakerAgent:
    def __init__(self, code_length=4, searchtype="DFS"):
        self.code_length = code_length
        self.searchtype = searchtype.upper()
        self.memory = {"possible_codes": self._generate_all_codes()}
        self.last_guess = None
        self.visited = set()

        if self.searchtype == 'DFS':
            self.container = []
            self.container.extend(self.memory["possible_codes"])
        else:  # BFS
            self.container = deque()
            self.container.extend(self.memory["possible_codes"])

    def _generate_all_codes(self):
        return [str(i).zfill(self.code_length) for i in range(10 ** self.code_length)]

    def get_feedback(self, guess, secret):
        correct_place = sum(g == s for g, s in zip(guess, secret))
        correct_digit = sum(min(guess.count(d), secret.count(d)) for d in set(guess)) - correct_place
        return correct_place, correct_digit

    def update_memory(self, feedback):
        if self.last_guess is None:
            return

        new_possible = []
        for code in self.memory["possible_codes"]:
            f = self.get_feedback(self.last_guess, code)
            if f == feedback:
                new_possible.append(code)

        self.memory["possible_codes"] = new_possible
        remaining = [c for c in self.memory["possible_codes"] if c not in self.visited]

        if self.searchtype == 'DFS':
            self.container = remaining[::-1]  # stack
        else:
            self.container = deque(remaining)  # queue

    def act(self):
        if not self.memory["possible_codes"]:
            return "No possible codes left!"

        if self.searchtype == 'DFS':
            self.last_guess = self.container.pop()
        else:  # BFS
            self.last_guess = self.container.popleft()

        self.visited.add(self.last_guess)
        return self.last_guess


class CodebreakerGUI:
    def __init__(self, master):
        self.master = master
        self.master.title("🔐 DFS & BFS Codebreaker")
        self.master.geometry("550x650")
        self.master.resizable(True, True)
        self.master.minsize(500, 600)
        
        # Color Palette - Purple/Pink gradient theme
        self.colors = {
            'bg': '#1a0933',           # Deep purple background
            'primary': '#6C3EB8',      # Rich purple
            'secondary': '#9D4EDD',    # Bright purple
            'accent': '#E0AAFF',       # Light purple
            'text': '#FFFFFF',         # White text
            'input_bg': '#2D1250',     # Dark purple for inputs
            'button': '#C77DFF',       # Vibrant purple
            'button_hover': '#E0AAFF', # Light purple hover
            'output_bg': '#240A4A',    # Dark output background
            'success': '#7B2CBF',      # Success purple
        }
        
        self.master.configure(bg=self.colors['bg'])
        
        # Main container with reduced padding
        main_frame = tk.Frame(master, bg=self.colors['bg'])
        main_frame.pack(fill='both', padx=20, pady=15)
        
        # Title
        title = tk.Label(
            main_frame, 
            text="🔐 CODEBREAKER AI 🔐",
            font=("Arial", 24, "bold"),
            fg=self.colors['accent'],
            bg=self.colors['bg']
        )
        title.pack(pady=(0, 8))
        
        # Subtitle
        subtitle = tk.Label(
            main_frame,
            text="Watch AI solve your secret code using search algorithms",
            font=("Arial", 10),
            fg=self.colors['secondary'],
            bg=self.colors['bg']
        )
        subtitle.pack(pady=(0, 15))
        
        # Input section frame
        input_frame = tk.Frame(main_frame, bg=self.colors['primary'], relief=tk.RAISED, bd=0)
        input_frame.pack(fill='x', pady=8)
        
        # Secret code label
        tk.Label(
            input_frame,
            text="Enter Your Secret 4-Digit Code",
            font=("Arial", 12, "bold"),
            fg=self.colors['text'],
            bg=self.colors['primary']
        ).pack(pady=(12, 5))
        
        # Secret entry with custom styling
        self.secret_entry = tk.Entry(
            input_frame,
            width=10,
            justify="center",
            font=("Arial", 22, "bold"),
            bg=self.colors['input_bg'],
            fg=self.colors['accent'],
            insertbackground=self.colors['accent'],
            relief=tk.FLAT,
            bd=5
        )
        self.secret_entry.pack(pady=(5, 12))
        
        # Search type section
        search_frame = tk.Frame(main_frame, bg=self.colors['primary'], relief=tk.RAISED, bd=0)
        search_frame.pack(fill='x', pady=8)
        
        tk.Label(
            search_frame,
            text="Choose Search Algorithm",
            font=("Arial", 12, "bold"),
            fg=self.colors['text'],
            bg=self.colors['primary']
        ).pack(pady=(12, 8))
        
        # Radio buttons frame
        radio_frame = tk.Frame(search_frame, bg=self.colors['primary'])
        radio_frame.pack(pady=(0, 12))
        
        self.searchvariable = tk.StringVar(value="DFS")
        
        # Custom radio buttons
        for text, value in [("DFS - Depth First", "DFS"), ("BFS - Breadth First", "BFS")]:
            rb = tk.Radiobutton(
                radio_frame,
                text=text,
                variable=self.searchvariable,
                value=value,
                font=("Arial", 11, "bold"),
                fg=self.colors['text'],
                bg=self.colors['primary'],
                selectcolor=self.colors['secondary'],
                activebackground=self.colors['primary'],
                activeforeground=self.colors['accent'],
                relief=tk.FLAT,
                bd=0
            )
            rb.pack(side='left', padx=15)
        
        # Start button with hover effect
        self.start_button = tk.Button(
            main_frame,
            text="🚀 START GAME",
            command=self.start_game,
            font=("Arial", 13, "bold"),
            bg=self.colors['button'],
            fg=self.colors['text'],
            activebackground=self.colors['button_hover'],
            activeforeground=self.colors['bg'],
            relief=tk.FLAT,
            bd=0,
            cursor="hand2",
            padx=25,
            pady=10
        )
        self.start_button.pack(pady=15)
        
        # Bind hover effects
        self.start_button.bind('<Enter>', lambda e: self.start_button.config(bg=self.colors['button_hover']))
        self.start_button.bind('<Leave>', lambda e: self.start_button.config(bg=self.colors['button']))
        
        # Output section
        output_label = tk.Label(
            main_frame,
            text="📊 GAME LOG",
            font=("Arial", 11, "bold"),
            fg=self.colors['accent'],
            bg=self.colors['bg']
        )
        output_label.pack(pady=(8, 5))
        
        # Output frame
        output_frame = tk.Frame(main_frame, bg=self.colors['secondary'], bd=2, relief=tk.RAISED)
        output_frame.pack(fill='both', expand=True, pady=5)
        
        # Text widget (NO FIXED HEIGHT)
        self.output = tk.Text(
            output_frame,
            state="disabled",
            font=("Courier New", 10, "bold"),
            bg=self.colors['output_bg'],
            fg=self.colors['accent'],
            insertbackground=self.colors['accent'],
            relief=tk.FLAT,
            bd=8,
            wrap="word"
        )

        self.output.pack(side="left", fill="both", expand=True)
        
        # Scrollbar
        scrollbar = tk.Scrollbar(output_frame, orient="vertical", command=self.output.yview)
        scrollbar.pack(side="right", fill="y")
        
        self.output.config(yscrollcommand=scrollbar.set)


    def start_game(self):
        secret = self.secret_entry.get().strip()

        if not (secret.isdigit() and len(secret) == 4):
            messagebox.showerror("Error", "Please enter a valid 4-digit number.")
            return


        searchtype = self.searchvariable.get()
        self.agent = CodebreakerAgent(code_length=4, searchtype=searchtype)
        self.step = 0
        self.secret = secret

        self.output.config(state="normal")
        self.output.delete(1.0, tk.END)
        self.output.insert(tk.END, f"╔══════════════════════════════════════╗\n")
        self.output.insert(tk.END, f"  Agent using {searchtype} to crack code...\n")
        self.output.insert(tk.END, f"╚══════════════════════════════════════╝\n\n")
        self.output.config(state="disabled")

        self.master.after(500, self.make_guess)

    def make_guess(self):
        guess = self.agent.act()
        if not isinstance(guess, str):
            self.show_message("❌ No possible codes left!")
            return

        feedback = self.agent.get_feedback(guess, self.secret)
        self.agent.update_memory(feedback)
        self.step += 1

        self.output.config(state="normal")
        
        self.output.insert(
            tk.END,
            f"Step {self.step}\n"
            f"  Guess     : {guess}\n"
            f"  Feedback  : ({feedback[0]}, {feedback[1]})\n\n"
        )
        
        self.output.config(state="disabled")
        self.output.see(tk.END)

        if feedback[0] == 4:
            messagebox.showinfo("🎉 SUCCESS!", f"Agent cracked your code '{self.secret}' in {self.step} steps!")
        elif self.step < 20:
            self.master.after(400, self.make_guess)
        else:
            messagebox.showinfo("⏱️ Time's Up", "Agent could not find the code within 20 steps.")

    def show_message(self, text):
        self.output.config(state="normal")
        self.output.insert(tk.END, text + "\n")
        self.output.config(state="disabled")
        self.output.see(tk.END)


root = tk.Tk()
app = CodebreakerGUI(root)
root.mainloop()